# Data preparation

In [1]:
URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'examples/fashion_mnist/data/'

In [2]:
import os
import urllib
import urllib.request

In [3]:
if not os.path.isfile(FILE):
    print(f"Downloading {URL} and saving as {FILE}...")
    urllib.request.urlretrieve(URL,FILE)

    from zipfile import ZipFile
    print("Unzipping images...")
    with ZipFile(FILE) as zip_images:
        zip_images.extractall(FOLDER)

# Data loading

In [4]:
import os

DIR = 'examples/fashion_mnist'
DATA_DIR = 'examples/fashion_mnist/data'
TRAIN_DIR = 'examples/fashion_mnist/data/train'
TEST_DIR = 'examples/fashion_mnist/data/test'

labels = os.listdir(TRAIN_DIR)

files = os.listdir(TRAIN_DIR + '/0')

In [5]:
import cv2
import numpy as np
image_data = cv2.imread(TRAIN_DIR + '/7/0002.png', cv2.IMREAD_UNCHANGED)

np.set_printoptions(linewidth=200)

In [6]:
def load_mnist_dataset(dataset,path):
    
    # Scan ll the directories and create a list of labels
    labels = os.listdir(os.path.join(path, dataset))
    
    X, y = [], []
    
    for label in labels:
        for file in os.listdir(os.path.join(path, dataset, label)):
            image = cv2.imread(os.path.join(path, dataset, label, file), cv2.IMREAD_UNCHANGED)
        
            X.append(image)
            y.append(label)
            
    # Convert the data to proper numpy arrays and return
    return np.array(X), np.array(y).astype('uint8')

def create_data_mnist(path):
    # Load both sets separately
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    # And return all the data
    return X, y, X_test, y_test

# Training

In [7]:
X,y,X_test,y_test = create_data_mnist(DATA_DIR)

In [8]:
# shuffle
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

In [9]:
# flatten for dense NN, from -1 to 1
X = (X.reshape(X.shape[ 0 ], - 1 ).astype(np.float32) - 127.5 ) / 127.5
X_test = (X_test.reshape(X_test.shape[ 0 ], - 1 ).astype(np.float32) -
127.5 ) / 127.5

assert -1 <= X.min(), X.max() <= 1 # Ensuring that scaling is correct
assert -1 <= X_test.min(), X_test.max() <= 1 

In [10]:
from nn.model import *

from nn.layer import *
from nn.activation import *

from nn.optimizer import *
from nn.loss import *
from nn.accuracy import *

# Instantiate the model
model = Model()
# Add layers
model.add(Layer_Dense(X.shape[ 1 ], 64 ))
model.add(Activation_ReLU())
model.add(Layer_Dense( 64 , 64 ))
model.add(Activation_ReLU())
model.add(Layer_Dense( 64 , 10 ))
model.add(Activation_Softmax())

# Set loss, optimizer and accuracy objects
model.set(
    loss = Loss_CategoricalCrossentropy(),
    optimizer = Optimizer_Adam( decay = 5e-5 ),
    accuracy = Accuracy_Categorical()
)

# Train the model
model.train(X, y, validation_data = (X_test, y_test), 
            epochs = 5 , batch_size = 128 , print_every = 100)

# Retrieve model parameters
parameters = model.get_parameters()

epoch: 1
step: 0, acc: 0.094, loss: 2.303 (data_loss: 2.303, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.758, loss: 0.720 (data_loss: 0.720, reg_loss: 0.000), lr: 0.0009950248756218907
step: 200, acc: 0.758, loss: 0.570 (data_loss: 0.570, reg_loss: 0.000), lr: 0.0009900990099009901
step: 300, acc: 0.781, loss: 0.566 (data_loss: 0.566, reg_loss: 0.000), lr: 0.0009852216748768474
step: 400, acc: 0.852, loss: 0.405 (data_loss: 0.405, reg_loss: 0.000), lr: 0.000980392156862745
step: 468, acc: 0.854, loss: 0.427 (data_loss: 0.427, reg_loss: 0.000), lr: 0.0009771350400625367
training, acc: 0.731, loss: 0.736 (data_loss: 0.736, reg_loss: 0.000), lr: 0.0009771350400625367
epoch: 2
step: 0, acc: 0.773, loss: 0.528 (data_loss: 0.528, reg_loss: 0.000), lr: 0.0009770873027505008
step: 100, acc: 0.805, loss: 0.440 (data_loss: 0.440, reg_loss: 0.000), lr: 0.000972337012008362
step: 200, acc: 0.891, loss: 0.354 (data_loss: 0.354, reg_loss: 0.000), lr: 0.0009676326866321544
step: 300, acc: 0.867, lo

In [11]:
# # New model
# # Instantiate the model
# model = Model()
# # Add layers
# model.add(Layer_Dense(X.shape[ 1 ], 128 ))
# model.add(Activation_ReLU())
# model.add(Layer_Dense( 128 , 128 ))
# model.add(Activation_ReLU())
# model.add(Layer_Dense( 128 , 10 ))
# model.add(Activation_Softmax())
# # Set loss and accuracy objects
# # We do not set optimizer object this time - there's no need to do it
# # as we won't train the model
# model.set(
# loss = Loss_CategoricalCrossentropy(),
# accuracy = Accuracy_Categorical()
# )
# # Set model with parameters instead of training it
# model.set_parameters(parameters)
# # Evaluate the model
# model.evaluate(X_test, y_test)

In [12]:
if not os.path.isfile(DIR + '/params'):
    model.save_parameters(DIR + '/params')

In [13]:
model = Model('basic', [X.shape[1], 128, 10])
model.set_parameters(parameters)
model.evaluate(X_test, y_test)

validation, acc: 0.859, loss: 0.394
